# --- Day 7: Recursive Circus ---

Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:
```
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
```
...then you would be able to recreate the structure of the towers that looks like this:
```
                gyxo
              /     
         ugml - ebii
       /      \     
      |         jptl
      |        
      |         pbga
     /        /
tknk --- padx - havc
     \        \
      |         qoyq
      |             
      |         ktlj
       \      /     
         fwft - cntj
              \     
                xhth
```

In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

In [1]:
with open("day7.txt") as f:
    stack = f.readlines()

In [2]:
nodes = [x.split(" ")[0] for x in stack]
weights = [int(x.split("(")[1].split(")")[0]) for x in stack]
children = [x.split(")")[1].strip(" -> ").strip("\n").split(", ") for x in stack]

In [7]:
bottom = set()
for node in nodes:
    for child in children:
        if node in child:
            bottom.add(node)

In [10]:
for node in nodes:
    if node not in bottom:
        print(node)

eugwuhl


--- Part Two ---

The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.

In the example above, this means that for ugml's disc to be balanced, gyxo, ebii, and jptl must all have the same weight, and they do: 61.

However, for tknk to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same:

ugml + (gyxo + ebii + jptl) = 68 + (61 + 61 + 61) = 251
padx + (pbga + havc + qoyq) = 45 + (66 + 66 + 66) = 243
fwft + (ktlj + cntj + xhth) = 72 + (57 + 57 + 57) = 243
As you can see, tknk's disc is unbalanced: ugml's stack is heavier than the other two. Even though the nodes above ugml are balanced, ugml itself is too heavy: it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced. If this change were made, its weight would be 60.

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?



In [11]:
data = {}
for i in range(len(nodes)):
    data[nodes[i]] = {"weight": weights[i], "children": children[i]}

In [12]:
def get_child_weights(parent, dict_of_parents):
    '''
    parent: name of key
    dict_of_parents: a dict of parent-child relationships
    example:
    get_child_weights('abc') # [1, 2, 3]
    '''
    children = dict_of_parents.get(parent).get('children')
    child_weight = []
    for child in children:
        child_weight.append(int(data[child.strip(",")]["weight"]))
    return(child_weight)

In [13]:
get_child_weights('byhatd', data)

[22, 225, 263]

In [14]:
def find_parent(node, nodes):
    for k in nodes:
        if node in nodes.get(k).get('children'):
            return k

In [15]:
child_nodes = {}
for k, v in data.items():
    if v['children'] == ['']:
        child_nodes[k] = v['weight']

In [16]:
parents = {}
for k, v in child_nodes.items():
    parent_node = find_parent(k, data)
    if parent_node in parents:
        parents[parent_node] += v
    else:
        parents[parent_node] = v + data.get(parent_node).get('weight')
parents['aodkqlr']

261

In [17]:
assert parents['aodkqlr'] == sum(get_child_weights('aodkqlr', data)) + data['aodkqlr']['weight']

In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame.from_dict(data=data, orient='index')

In [20]:
df['parent'] = [find_parent(x, data) for x in data.keys()]

In [21]:
df.head()

,weight,children,parent
aaeeqh,68,"[nmyem, fserz]",ozvtq
aaroa,121,[],hgynpc
acgef,17,[],fhbjovx
acrly,75,[],wlfngqq
actlu,15,[],irgkfsa


In [22]:
df.groupby('parent').mean().head()

,weight
parent,
aaeeqh,724.500000
adyni,130.666667
aenvjl,44.666667
aflzz,47.500000
aftndu,50.000000


In [23]:
get_child_weights('zuhkp', data)

[50, 44, 70]

In [24]:
for k in data.keys():
    if k not in parents and k not in child_nodes:
        parent_node = find_parent(k, data)
        if parent_node in parents:
            parents[parent_node] += data.get(k).get('weight')
        else:
            parents[parent_node] = data.get(k).get('weight') + data.get(parent_node).get('weight')
print(len(parents))
parents['eugwuhl']

420


58907

In [25]:
sum(get_child_weights('eugwuhl', data)) + data['eugwuhl'].get('weight')

75435

In [26]:
parent_df = pd.DataFrame.from_dict(parents, orient='index')

In [27]:
parent_df.head()

,0
ozvtq,314
hgynpc,372
fhbjovx,1713
irgkfsa,155
kqdus,188


In [28]:
print(data['zuhkp'])
# parents['zuhkp']
get_child_weights('zuhkp', data)

{'weight': 1365, 'children': ['bjdlb', 'ihimta', 'ffejzo']}


[50, 44, 70]

In [29]:
for child in data['zuhkp']['children']:
    print("in child_nodes", child in child_nodes, "in parents", child in parents)

in child_nodes False in parents True
in child_nodes False in parents True
in child_nodes False in parents True


In [30]:
'zuhkp' not in parents and 'zuhkp' not in child_nodes

True

In [31]:
find_parent('zuhkp', data)

'qrxso'

In [32]:
parents['qrxso']

2841

In [33]:
'zuhkp' in data.keys()

True

In [34]:
data.get('aaroa').get('children') == ['']

True

In [35]:
def find_all_children(node, dict_of_nodes):
    children = []
    for child in dict_of_nodes.get(node).get('children'):
        if child == '':
            continue
        else:
            children.extend()
    return children

In [36]:
def node_weight(node, dict_of_nodes):
    weight = dict_of_nodes.get(node).get('weight')
    for child in dict_of_nodes.get(node).get('children'):
        if child == '':
            pass
        else:
            weight += node_weight(child, dict_of_nodes)
    return weight

In [37]:
node_weight('zuhkp', data)

1833

In [38]:
weights = {}
for key in data:
    weights[key] = {'weight': node_weight(key, data)}

In [39]:
weight_df = pd.DataFrame.from_dict(weights, orient='index')
weight_df.head()

,weight
aaeeqh,248
aaroa,121
acgef,17
acrly,75
actlu,15


In [40]:
weight_df['name'] = weight_df.index

In [41]:
weight_df['parent'] = weight_df.name.apply(lambda x: find_parent(x, data))

In [42]:
weight_df[weight_df.weight == weight_df.weight.max()]

,weight,name,parent
eugwuhl,338014,eugwuhl,None


In [43]:
same_weights = weight_df[['parent','weight']].groupby('parent').min() == weight_df[['parent', 'weight']].groupby('parent').max()

In [44]:
same_weights[same_weights.weight == False]

,weight
parent,
eugwuhl,False
hmgrlpj,False
smaygo,False


In [45]:
weight_df['node_weight'] = weight_df.name.apply(lambda x: data[x]['weight'])

In [46]:
weight_df[weight_df.parent == 'eugwuhl']

,weight,name,parent,node_weight
avpklqy,48284,avpklqy,eugwuhl,68
bdohoaa,48284,bdohoaa,eugwuhl,18440
hgizeb,48284,hgizeb,eugwuhl,5324
pvvbn,48284,pvvbn,eugwuhl,8732
smaygo,48292,smaygo,eugwuhl,4616
tchfafn,48284,tchfafn,eugwuhl,31649
tytbgx,48284,tytbgx,eugwuhl,6588


In [47]:
weight_df[weight_df.parent == 'smaygo']

,weight,name,parent,node_weight
fbnbt,14556,fbnbt,smaygo,5262
hmgrlpj,14564,hmgrlpj,smaygo,66
nfdvsc,14556,nfdvsc,smaygo,4824


In [48]:
weight_df[weight_df.parent == 'hmgrlpj']

,weight,name,parent,node_weight
ahrfh,2070,ahrfh,hmgrlpj,2040
drjmjug,2078,drjmjug,hmgrlpj,428
luasrvp,2070,luasrvp,hmgrlpj,1476
nigdlq,2070,nigdlq,hmgrlpj,846
omytneg,2070,omytneg,hmgrlpj,1366
pnqrgfk,2070,pnqrgfk,hmgrlpj,1707
tbfce,2070,tbfce,hmgrlpj,5


'drjmjug' needs to be decreased by 8 to 420

That's the right answer! You are one gold star closer to debugging the printer.